# 네이버 영화리뷰 감정 분석 문제에 SentencePiece 적용해 보기

In [1]:
import pandas as pd
import sentencepiece as spm
import tensorflow as tf
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import time

## 1. 데이터 로드 및 전처리

In [2]:
# 네이버 영화 리뷰 데이터셋을 로드하고 전처리하는 과정
train_txt = '/Users/jian_lee/Desktop/aiffel/data/text_process/Data/ratings_train.txt'
test_txt = '/Users/jian_lee/Desktop/aiffel/data/text_process/Data/ratings_test.txt'

def load_data(file):
    """
    텍스트 파일을 로드하여 DataFrame으로 변환하는 함수
    - 첫 번째 컬럼은 id, 두 번째 컬럼은 리뷰 내용(document), 세 번째 컬럼은 라벨(label)이다.
    - 첫 줄은 헤더이므로 제거하고, id 컬럼은 사용하지 않는다.
    - 결측치를 제거하고 데이터를 랜덤하게 섞는다.
    """
    df = pd.read_csv(file, delimiter='\t', names=['id', 'document', 'label'], skiprows=1)
    df.dropna(inplace=True)  # 결측치 제거
    df = df.sample(frac=1).reset_index(drop=True)  # 데이터 랜덤 섞기
    return df

# 훈련 및 테스트 데이터 로드
train_data = load_data(train_txt)
test_data = load_data(test_txt)

print(f"Train Data: {train_data.shape}")
print(f"Test Data: {test_data.shape}")

Train Data: (149995, 3)
Test Data: (49997, 3)


## 2. 토크나이저 클래스 정의

In [3]:
# 2.1 SentencePiece 토크나이저 클래스
class SentencePieceTokenizer:
    def __init__(self, vocab_size=8000):
        self.vocab_size = vocab_size
        self.sp_model_prefix = 'spm_review'
        self.sp_model_path = f'{self.sp_model_prefix}.model'
        self.sp = None
        self.vocab_size_actual = None
        self.tokenization_time = 0
    
    def train(self, texts):
        """SentencePiece 모델 학습"""
        start_time = time.time()
        
        # 훈련 데이터 준비
        with open('spm_input.txt', 'w', encoding='utf-8') as f:
            for text in texts:
                f.write(text + '\n')
        
        # 모델 훈련
        spm.SentencePieceTrainer.Train(
            input='spm_input.txt',
            model_prefix=self.sp_model_prefix,
            vocab_size=self.vocab_size,
            model_type='bpe',
            character_coverage=0.9995,
        )
        
        # 모델 로드
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(self.sp_model_path)
        self.vocab_size_actual = self.sp.GetPieceSize()
        
        self.tokenization_time = time.time() - start_time
        print(f"SentencePiece 모델 학습 완료 (시간: {self.tokenization_time:.2f}초)")
    
    def load(self):
        """학습된 SentencePiece 모델 로드"""
        if os.path.exists(self.sp_model_path):
            self.sp = spm.SentencePieceProcessor()
            self.sp.Load(self.sp_model_path)
            self.vocab_size_actual = self.sp.GetPieceSize()
            return True
        return False
    
    def tokenize(self, texts):
        """텍스트 리스트를 토큰화"""
        start_time = time.time()
        tokenized = [self.sp.EncodeAsIds(text) for text in texts]
        self.tokenization_time = time.time() - start_time
        return tokenized
    
    def detokenize(self, token_ids):
        """토큰 ID 리스트를 텍스트로 변환"""
        if isinstance(token_ids[0], list):
            return [self.sp.DecodeIds(ids) for ids in token_ids]
        return self.sp.DecodeIds(token_ids)

# 2.2 KoNLPy Okt 토크나이저 클래스
class OktTokenizer:
    def __init__(self):
        self.okt = Okt()
        self.word_to_index = {}
        self.index_to_word = {}
        self.vocab_size = 0
        self.tokenization_time = 0
    
    def train(self, texts, min_freq=2):
        """어휘 사전 구축"""
        start_time = time.time()
        
        # Okt 토크나이저 초기화
        self.okt = Okt()
        
        # 단어 빈도수 계산
        word_counts = {}
        for text in texts:
            tokens = self.okt.morphs(text, stem=True)  # 형태소 분석 (스테밍 적용)
            for token in tokens:
                if token not in word_counts:
                    word_counts[token] = 0
                word_counts[token] += 1
        
        # 최소 빈도수 이상 단어만 사전에 추가
        vocab = ['<PAD>', '<UNK>']  # 패딩과 알 수 없는 단어를 위한 특수 토큰
        vocab.extend([word for word, count in word_counts.items() if count >= min_freq])
        
        # 단어-인덱스 매핑 구축
        self.word_to_index = {word: i for i, word in enumerate(vocab)}
        self.index_to_word = {i: word for i, word in enumerate(vocab)}
        self.vocab_size = len(vocab)
        
        self.tokenization_time = time.time() - start_time
        print(f"Okt 어휘 사전 구축 완료: {self.vocab_size} 단어 (시간: {self.tokenization_time:.2f}초)")
    
    def tokenize(self, texts):
        """텍스트 리스트를 토큰화하여 인덱스 리스트로 변환"""
        start_time = time.time()
        tokenized = []
        for text in texts:
            tokens = self.okt.morphs(text, stem=True)
            indices = [self.word_to_index.get(token, 1) for token in tokens]  # 1은 <UNK> 토큰
            tokenized.append(indices)
        
        self.tokenization_time = time.time() - start_time
        return tokenized
    
    def detokenize(self, token_ids):
        """토큰 ID 리스트를 텍스트로 변환"""
        if isinstance(token_ids[0], list):
            texts = []
            for ids in token_ids:
                tokens = [self.index_to_word.get(idx, '<UNK>') for idx in ids if idx != 0]  # 0은 <PAD> 토큰
                texts.append(' '.join(tokens))
            return texts
        else:
            tokens = [self.index_to_word.get(idx, '<UNK>') for idx in token_ids if idx != 0]
            return ' '.join(tokens)

# 2.3 KoNLPy Kkma 토크나이저 클래스
class KkmaTokenizer:
    def __init__(self):
        from konlpy.tag import Kkma
        self.kkma = Kkma()
        self.word_to_index = {}
        self.index_to_word = {}
        self.vocab_size = 0
        self.tokenization_time = 0
    
    def train(self, texts, min_freq=2):
        """어휘 사전 구축"""
        start_time = time.time()
        
        # Kkma 토크나이저 초기화
        from konlpy.tag import Kkma
        self.kkma = Kkma()
        
        # 단어 빈도수 계산
        word_counts = {}
        for text in texts:
            tokens = self.kkma.morphs(text)  # 형태소 분석
            for token in tokens:
                if token not in word_counts:
                    word_counts[token] = 0
                word_counts[token] += 1
        
        # 최소 빈도수 이상 단어만 사전에 추가
        vocab = ['<PAD>', '<UNK>']  # 패딩과 알 수 없는 단어를 위한 특수 토큰
        vocab.extend([word for word, count in word_counts.items() if count >= min_freq])
        
        # 단어-인덱스 매핑 구축
        self.word_to_index = {word: i for i, word in enumerate(vocab)}
        self.index_to_word = {i: word for i, word in enumerate(vocab)}
        self.vocab_size = len(vocab)
        
        self.tokenization_time = time.time() - start_time
        print(f"Kkma 어휘 사전 구축 완료: {self.vocab_size} 단어 (시간: {self.tokenization_time:.2f}초)")
    
    def tokenize(self, texts):
        """텍스트 리스트를 토큰화하여 인덱스 리스트로 변환"""
        start_time = time.time()
        tokenized = []
        for text in texts:
            tokens = self.kkma.morphs(text)
            indices = [self.word_to_index.get(token, 1) for token in tokens]  # 1은 <UNK> 토큰
            tokenized.append(indices)
        
        self.tokenization_time = time.time() - start_time
        return tokenized
    
    def detokenize(self, token_ids):
        """토큰 ID 리스트를 텍스트로 변환"""
        if isinstance(token_ids[0], list):
            texts = []
            for ids in token_ids:
                tokens = [self.index_to_word.get(idx, '<UNK>') for idx in ids if idx != 0]  # 0은 <PAD> 토큰
                texts.append(' '.join(tokens))
            return texts
        else:
            tokens = [self.index_to_word.get(idx, '<UNK>') for idx in token_ids if idx != 0]
            return ' '.join(tokens)

## 3. 감정 분석 모델 정의

In [4]:
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate=0.5):
        super(SentimentLSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim, 
            hidden_dim, 
            num_layers=n_layers, 
            bidirectional=True, 
            dropout=dropout_rate if n_layers > 1 else 0,
            batch_first=True
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, text):
        embedded = self.embedding(text)
        packed_output, (hidden, cell) = self.lstm(embedded)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(hidden)
        output = self.fc(hidden)
        return self.sigmoid(output)

## 4. 학습 및 평가 함수

In [5]:
def train_epoch(model, data_loader, optimizer, criterion, device):
    """한 에폭 동안 모델을 학습시키는 함수"""
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    for texts, labels in data_loader:
        texts = texts.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        predicted_labels = (predictions >= 0.5).int()
        correct_predictions += (predicted_labels == labels.int()).sum().item()
        total_predictions += labels.size(0)
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / total_predictions
    
    return avg_loss, accuracy

def evaluate(model, data_loader, criterion, device):
    """모델을 평가하는 함수"""
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for texts, labels in data_loader:
            texts = texts.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            
            predictions = model(texts)
            loss = criterion(predictions, labels)
            
            total_loss += loss.item()
            predicted_labels = (predictions >= 0.5).int()
            correct_predictions += (predicted_labels == labels.int()).sum().item()
            total_predictions += labels.size(0)
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / total_predictions
    
    return avg_loss, accuracy

## 5. 실험 실행 함수

In [6]:
def run_experiment(tokenizer_name, tokenizer, train_data, test_data, max_len=100, batch_size=64, epochs=5):
    """특정 토크나이저를 사용한 실험을 수행하는 함수"""
    print(f"\n{'='*50}")
    print(f"실험: {tokenizer_name} 토크나이저 사용")
    print(f"{'='*50}")
    
    # 1. 토크나이저 훈련 또는 로드
    if tokenizer_name == 'SentencePiece':
        if not tokenizer.load():
            print("SentencePiece 모델 훈련 시작...")
            tokenizer.train(train_data['document'])
    else:
        print(f"{tokenizer_name} 토크나이저 준비 시작...")
        tokenizer.train(train_data['document'])
    
    # 2. 데이터 토큰화
    print(f"{tokenizer_name} 토큰화 시작...")
    train_texts = tokenizer.tokenize(train_data['document'])
    test_texts = tokenizer.tokenize(test_data['document'])
    
    # 토큰화 소요 시간 기록
    print(f"토큰화 소요 시간: {tokenizer.tokenization_time:.2f}초")
    
    # 3. 패딩 처리
    train_texts = pad_sequences(train_texts, maxlen=max_len, padding='post')
    test_texts = pad_sequences(test_texts, maxlen=max_len, padding='post')
    
    # 4. PyTorch 데이터셋 및 데이터로더 생성
    train_texts_tensor = torch.tensor(train_texts, dtype=torch.long)
    train_labels_tensor = torch.tensor(train_data['label'].values, dtype=torch.long)
    test_texts_tensor = torch.tensor(test_texts, dtype=torch.long)
    test_labels_tensor = torch.tensor(test_data['label'].values, dtype=torch.long)
    
    # 검증 데이터 분리
    train_size = int(0.8 * len(train_texts_tensor))
    indices = torch.randperm(len(train_texts_tensor))
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    train_dataset = TensorDataset(
        train_texts_tensor[train_indices], 
        train_labels_tensor[train_indices]
    )
    val_dataset = TensorDataset(
        train_texts_tensor[val_indices], 
        train_labels_tensor[val_indices]
    )
    test_dataset = TensorDataset(test_texts_tensor, test_labels_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    # 5. 모델 설정
    vocab_size = tokenizer.vocab_size_actual if hasattr(tokenizer, 'vocab_size_actual') else tokenizer.vocab_size
    vocab_size = vocab_size + 1 if tokenizer_name == 'SentencePiece' else vocab_size  # SentencePiece는 <unk> 토큰 포함
    
    embedding_dim = 300
    hidden_dim = 256
    n_layers = 2
    dropout_rate = 0.5
    
    device = torch.device("mps" if torch.backends.mps.is_available() 
                      else ("cuda" if torch.cuda.is_available() else "cpu"))
    model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, n_layers, dropout_rate).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 6. 학습 실행
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_loss = float('inf')
    best_model_path = f'best_sentiment_model_{tokenizer_name}.pt'
    
    print(f"{tokenizer_name} 모델 학습 시작...")
    start_time = time.time()
    
    for epoch in range(epochs):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Epoch {epoch+1}: Best model saved with validation loss: {val_loss:.4f}")
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"\tTrain Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
        print(f"\tVal Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
    
    training_time = time.time() - start_time
    print(f"학습 소요 시간: {training_time:.2f}초")
    
    # 7. 최적 모델 로드 및 테스트
    model.load_state_dict(torch.load(best_model_path))
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"테스트 결과: Loss: {test_loss:.4f} | Acc: {test_acc*100:.2f}%")
    
    # 8. 시각화 데이터 및 성능 메트릭 반환
    metrics = {
        'name': tokenizer_name,
        'vocab_size': vocab_size,
        'tokenization_time': tokenizer.tokenization_time,
        'training_time': training_time,
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies,
        'test_loss': test_loss,
        'test_accuracy': test_acc,
    }
    
    return model, tokenizer, metrics

## 6. 예측 및 평가 함수

In [7]:
def predict_sentiment(text, model, tokenizer, device, max_len=100):
    """새로운 텍스트의 감정을 예측하는 함수"""
    model.eval()
    
    # 텍스트 토큰화
    tokens = tokenizer.tokenize([text])[0]
    
    # 패딩 추가
    if len(tokens) < max_len:
        tokens = tokens + [0] * (max_len - len(tokens))
    else:
        tokens = tokens[:max_len]
    
    # 텐서로 변환
    tokens_tensor = torch.tensor(tokens).unsqueeze(0).to(device)
    
    # 예측
    with torch.no_grad():
        prediction = model(tokens_tensor)
    
    # 결과 반환
    prob = prediction.item()
    sentiment = 1 if prob >= 0.5 else 0
    
    return sentiment, prob

def compare_predictions(models_tokenizers, example_texts):
    """여러 모델-토크나이저 쌍에 대해 예제 텍스트 예측 비교"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print("\n예제 텍스트 예측 비교:")
    for i, text in enumerate(example_texts, 1):
        print(f"\n[예제 {i}] {text}")
        
        for model, tokenizer, tokenizer_name in models_tokenizers:
            sentiment, prob = predict_sentiment(text, model, tokenizer, device)
            print(f"{tokenizer_name:15s}: {'긍정' if sentiment == 1 else '부정'} (확률: {prob:.4f})")

## 7. 시각화 함수

In [8]:
def plot_comparison(metrics_list):
    """토크나이저별 성능 비교 시각화"""
    plt.figure(figsize=(20, 15))
    plt.rcParams['font.family'] = 'AppleGothic, Arial'  # 한글 폰트 설정
    
    # 토크나이저별 색상 설정
    colors = {
        'SentencePiece': '#3498DB',  # 파랑
        'Okt': '#E74C3C',           # 빨강
        'Kkma': '#2ECC71'           # 초록
    }
    
    markers = {
        'SentencePiece': 'o',  # 원형
        'Okt': 's',           # 사각형
        'Kkma': '^'           # 삼각형
    }
    
    # 1. 정확도 비교
    plt.subplot(2, 2, 1)
    for metrics in metrics_list:
        name = metrics['name']
        plt.plot(metrics['train_accuracies'], 
                 label=f"{name} (학습)", 
                 color=colors[name], 
                 marker=markers[name],
                 markersize=8)
        plt.plot(metrics['val_accuracies'], 
                 label=f"{name} (검증)", 
                 color=colors[name], 
                 linestyle='--',
                 marker=markers[name],
                 markersize=6,
                 alpha=0.7)
    
    plt.xlabel('에폭', fontsize=12)
    plt.ylabel('정확도', fontsize=12)
    plt.title('토크나이저별 학습 및 검증 정확도 비교', fontsize=16, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.ylim([0.7, 1.0])  # 정확도 범위 조정
    
    # 2. 손실 비교
    plt.subplot(2, 2, 2)
    for metrics in metrics_list:
        name = metrics['name']
        plt.plot(metrics['train_losses'], 
                 label=f"{name} (학습)", 
                 color=colors[name], 
                 marker=markers[name],
                 markersize=8)
        plt.plot(metrics['val_losses'], 
                 label=f"{name} (검증)", 
                 color=colors[name], 
                 linestyle='--',
                 marker=markers[name],
                 markersize=6,
                 alpha=0.7)
    
    plt.xlabel('에폭', fontsize=12)
    plt.ylabel('손실', fontsize=12)
    plt.title('토크나이저별 학습 및 검증 손실 비교', fontsize=16, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # 3. 테스트 정확도 비교 (막대 그래프)
    plt.subplot(2, 2, 3)
    names = [m['name'] for m in metrics_list]
    test_accs = [m['test_accuracy'] * 100 for m in metrics_list]
    bar_colors = [colors[name] for name in names]
    
    bars = plt.bar(names, test_accs, color=bar_colors, width=0.6, edgecolor='black', linewidth=1.5, alpha=0.8)
    plt.xlabel('토크나이저', fontsize=12)
    plt.ylabel('정확도 (%)', fontsize=12)
    plt.title('테스트 정확도 비교', fontsize=16, fontweight='bold')
    plt.ylim([85, 100])  # Y축 범위 조정
    
    # 정확도 수치 표시
    for bar, acc in zip(bars, test_accs):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # 4. 처리 시간 비교 (막대 그래프)
    plt.subplot(2, 2, 4)
    tokenization_times = [m['tokenization_time'] for m in metrics_list]
    training_times = [m['training_time'] for m in metrics_list]
    
    x = np.arange(len(names))
    width = 0.35
    
    # 토크나이저 시간은 더 투명하게, 학습 시간은 더 진하게
    plt.bar(x - width/2, tokenization_times, width, 
            label='토큰화 시간 (초)', alpha=0.7, 
            color=[colors[name] for name in names],
            edgecolor='black', linewidth=1.5, hatch='/')
            
    plt.bar(x + width/2, training_times, width, 
            label='학습 시간 (초)', alpha=0.9, 
            color=[colors[name] for name in names],
            edgecolor='black', linewidth=1.5)
    
    plt.xlabel('토크나이저', fontsize=12)
    plt.ylabel('시간 (초)', fontsize=12)
    plt.title('처리 시간 비교', fontsize=16, fontweight='bold')
    plt.xticks(x, names, fontsize=12)
    plt.legend(fontsize=10, loc='upper left')
    
    # 시간 수치 표시 (초 단위)
    for i, time_val in enumerate(tokenization_times):
        plt.text(i - width/2, time_val + 5, f'{time_val:.1f}초', ha='center', va='bottom', fontsize=10)
    for i, time_val in enumerate(training_times):
        plt.text(i + width/2, time_val + 5, f'{time_val:.1f}초', ha='center', va='bottom', fontsize=10)
    
    # 5. 추가: 어휘 크기 비교 (별도의 그래프로)
    plt.figure(figsize=(12, 6))
    vocab_sizes = [m['vocab_size'] for m in metrics_list]
    bar_colors = [colors[name] for name in names]
    
    bars = plt.bar(names, vocab_sizes, color=bar_colors, width=0.6, edgecolor='black', linewidth=1.5)
    plt.xlabel('토크나이저', fontsize=14)
    plt.ylabel('어휘 크기', fontsize=14)
    plt.title('토크나이저별 어휘 크기 비교', fontsize=18, fontweight='bold')
    
    # 단어 수 표시
    for bar, size in zip(bars, vocab_sizes):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                f'{size:,}', ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('tokenizer_vocab_comparison.png')
    
    # 원래 비교 그래프로 돌아가기
    plt.figure(1)
    plt.tight_layout()
    plt.savefig('tokenizer_comparison.png')
    plt.show()
    
# ✅ 토큰화 예시 시각화 
def visualize_tokenization_examples(tokenizers):
    """각 토크나이저별 토큰화 결과 비교 시각화"""
    example_sentences = [
        "이 영화는 정말 재미있었어요. 배우들의 연기도 훌륭했습니다.",
        "스토리가 너무 뻔해서 지루했어요."
    ]
    
    plt.figure(figsize=(15, 10))
    plt.rcParams['font.family'] = 'AppleGothic, Arial'  # 한글 폰트 설정
    
    colors = {
        'SentencePiece': '#3498DB',  # 파랑
        'Okt': '#E74C3C',           # 빨강
        'Kkma': '#2ECC71'           # 초록
    }
    
    # 각 문장에 대해
    for idx, sentence in enumerate(example_sentences):
        plt.subplot(len(example_sentences), 1, idx+1)
        
        y_positions = []
        y_ticks = []
        
        # 각 토크나이저별로 토큰화 결과 시각화
        for i, (name, tokenizer) in enumerate(tokenizers.items()):
            # 토큰화
            if name == 'SentencePiece':
                tokens = tokenizer.sp.EncodeAsPieces(sentence)
            elif name == 'Okt':
                tokens = tokenizer.okt.morphs(sentence, stem=True)
            else:  # Kkma
                tokens = tokenizer.kkma.morphs(sentence)
            
            # 토큰별 위치 계산
            x_positions = [0]
            for token in tokens:
                x_positions.append(x_positions[-1] + len(token) + 1)
            x_positions = x_positions[:-1]  # 마지막 위치 제거
            
            y_position = i * 2  # 각 토크나이저는 2칸씩 간격
            y_positions.append(y_position)
            y_ticks.append(y_position)
            
            # 토큰 시각화
            for j, (token, x) in enumerate(zip(tokens, x_positions)):
                plt.text(x, y_position, token, 
                         bbox=dict(facecolor=colors[name], alpha=0.3, edgecolor=colors[name], pad=5),
                         fontsize=10)
                
                # 토큰 사이에 화살표 연결
                if j < len(tokens) - 1:
                    next_x = x_positions[j+1]
                    plt.arrow(x + len(token) + 0.5, y_position, 
                              next_x - x - len(token) - 1, 0,
                              head_width=0.1, head_length=0.3, fc=colors[name], ec=colors[name], alpha=0.6)
        
        # 원본 문장 표시
        plt.text(0, max(y_positions) + 2, f"원문: {sentence}", fontsize=12, fontweight='bold')
        
        # 축 설정
        plt.yticks(y_ticks, list(tokenizers.keys()))
        plt.xticks([])
        plt.grid(axis='y', linestyle='--', alpha=0.3)
        plt.xlim(-1, max([len(sentence) * 1.5 for sentence in example_sentences]))
        plt.ylim(-1, max(y_positions) + 3)
        plt.title(f'문장 {idx+1} 토큰화 결과 비교', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('tokenization_example.png')
    plt.show()

In [9]:
def main():
    """메인 실행 함수: 전체 실험 실행"""
    # 샘플 수 제한 (테스트용)
    # 전체 데이터셋으로 학습할 때는 주석 처리
    # train_data_sample = train_data.sample(n=10000).reset_index(drop=True)
    # test_data_sample = test_data.sample(n=2000).reset_index(drop=True)
    
    # 전체 데이터셋 사용 (시간 단축을 위해 일부만 사용할 수도 있음)
    train_data_sample = train_data
    test_data_sample = test_data
    
    print("\n" + "="*80)
    print("📊 SentencePiece, Okt, Kkma 토크나이저 성능 비교 실험 시작")
    print("="*80)
    
    # 1. SentencePiece 실험
    sp_tokenizer = SentencePieceTokenizer(vocab_size=8000)
    sp_model, sp_tokenizer, sp_metrics = run_experiment(
        'SentencePiece', 
        sp_tokenizer, 
        train_data_sample, 
        test_data_sample,
        epochs=5
    )
    
    # 2. KoNLPy Okt 실험
    okt_tokenizer = OktTokenizer()
    okt_model, okt_tokenizer, okt_metrics = run_experiment(
        'Okt', 
        okt_tokenizer, 
        train_data_sample, 
        test_data_sample,
        epochs=5
    )
    
    # 3. KoNLPy Kkma 실험
    kkma_tokenizer = KkmaTokenizer()
    kkma_model, kkma_tokenizer, kkma_metrics = run_experiment(
        'Kkma', 
        kkma_tokenizer, 
        train_data_sample, 
        test_data_sample,
        epochs=5
    )
    
    # 4. 결과 비교 및 시각화
    metrics_list = [sp_metrics, okt_metrics, kkma_metrics]
    plot_comparison(metrics_list)
    
    # 5. 토큰화 예시 시각화
    tokenizers = {
        'SentencePiece': sp_tokenizer,
        'Okt': okt_tokenizer,
        'Kkma': kkma_tokenizer
    }
    visualize_tokenization_examples(tokenizers)
    
    # 6. 예제 텍스트 예측 비교
    example_texts = [
        "이 영화는 정말 재미있었어요. 배우들의 연기도 훌륭했고 스토리도 좋았습니다.",
        "시간 낭비였습니다. 스토리도 엉망이고 배우들의 연기도 별로였어요.",
        "그럭저럭 볼만했어요. 특별히 좋지도 나쁘지도 않았습니다.",
        "연출은 좋았지만 스토리가 너무 뻔해서 몰입이 안됐어요.",
        "배우들의 연기는 훌륭했지만 너무 지루했어요."
    ]
    
    models_tokenizers = [
        (sp_model, sp_tokenizer, 'SentencePiece'),
        (okt_model, okt_tokenizer, 'Okt'),
        (kkma_model, kkma_tokenizer, 'Kkma')
    ]
    
    compare_predictions(models_tokenizers, example_texts)
    
    # 7. 성능 요약 출력
    print("\n성능 요약:")
    print("-" * 50)
    for metrics in metrics_list:
        print(f"📌 {metrics['name']} 토크나이저:")
        print(f"  • 어휘 크기: {metrics['vocab_size']:,} 단어")
        print(f"  • 토큰화 시간: {metrics['tokenization_time']:.2f}초")
        print(f"  • 학습 시간: {metrics['training_time']:.2f}초")
        print(f"  • 테스트 정확도: {metrics['test_accuracy']*100:.2f}%")
        print("-" * 50)
    
    print("\n실험이 모두 완료되었습니다. 결과는 PNG 파일로 저장되었습니다.")

# 메인 함수 실행
if __name__ == "__main__":
    main()


📊 SentencePiece, Okt, Kkma 토크나이저 성능 비교 실험 시작

실험: SentencePiece 토크나이저 사용
SentencePiece 토큰화 시작...
토큰화 소요 시간: 0.58초
SentencePiece 모델 학습 시작...
Epoch 1: Best model saved with validation loss: 0.3466
Epoch 1/5
	Train Loss: 0.4286 | Train Acc: 79.74%
	Val Loss: 0.3466 | Val Acc: 84.49%
Epoch 2: Best model saved with validation loss: 0.3314
Epoch 2/5
	Train Loss: 0.3017 | Train Acc: 87.11%
	Val Loss: 0.3314 | Val Acc: 85.58%
Epoch 3: Best model saved with validation loss: 0.3271
Epoch 3/5
	Train Loss: 0.2406 | Train Acc: 90.03%
	Val Loss: 0.3271 | Val Acc: 85.92%
Epoch 4/5
	Train Loss: 0.1764 | Train Acc: 92.87%
	Val Loss: 0.3868 | Val Acc: 85.67%
Epoch 5/5
	Train Loss: 0.1239 | Train Acc: 95.16%
	Val Loss: 0.4298 | Val Acc: 85.52%
학습 소요 시간: 715.33초
테스트 결과: Loss: 0.3295 | Acc: 85.79%

실험: Okt 토크나이저 사용
Okt 토크나이저 준비 시작...
Okt 어휘 사전 구축 완료: 27309 단어 (시간: 243.31초)
Okt 토큰화 시작...
토큰화 소요 시간: 89.61초
Okt 모델 학습 시작...
Epoch 1: Best model saved with validation loss: 0.3401
Epoch 1/5
	Train Loss: 0.4014 |

java.lang.OutOfMemoryError: java.lang.OutOfMemoryError: Java heap space